In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home +
                          "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content)
category_color = json.loads(
    requests.get(eco_git_home +
                 "guidelines/colors/eco-category-color.json").content)
hue_color = json.loads(
    requests.get(eco_git_home +
                 "guidelines/colors/eco-single-hue-color.json").content)
mhue_color = json.loads(
    requests.get(eco_git_home +
                 "guidelines/colors/eco-multi-hue-color.json").content)
div_color = json.loads(
    requests.get(eco_git_home +
                 "guidelines/colors/eco-diverging-color.json").content)
config = json.loads(
    requests.get(eco_git_home +
                 "guidelines/charts/eco-global-config.json").content)
height = config["height"]
width = config["width"]
uid, height, width

('', 300, 500)

# Fig 1

In [6]:
df = pd.read_csv('raw/f1.csv')
df2 = pd.read_csv('raw/f2b.csv', header=None)
df = df.join(df2[1]).rename(columns={1: 'deals'})


In [7]:
f = "fig1_deals"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f1.replace("/data/", "/visualisation/").replace(".csv", ".json")))
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Q,value,deals
0,2018Q1,38.9,1124.324324
1,2018Q2,35.9,977.777778
2,2018Q3,30.1,987.387387
3,2018Q4,43.7,941.741742
4,2019Q1,26.1,1061.861862


In [9]:
base = alt.Chart(f1).encode(x=alt.X(
    "Q:N",
    sort=[],
    axis=alt.Axis(
        # grid=False,
        title='',
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domainColor=colors["eco-gray"],
        tickCount=10,
        ticks=False,
        # orient="top",
        labelAngle=0,
        # labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
        labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4):slice(datum.value,4)",
    ),
))
line = base.mark_line(color=colors["eco-turquiose"]).encode(y=alt.Y(
    "deals:Q",
    sort=[],
    axis=alt.Axis(
        grid=False,
        title="Deal count",
        titleX=5,
        titleY=-5,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="right",
        labelColor=colors["eco-turquiose"],
        titleColor=colors["eco-turquiose"],
        tickColor=colors["eco-turquiose"],
        domainColor=colors["eco-turquiose"],
    ),
    # scale=alt.Scale(domain=[-0.16, 0.06]),
))
bars = base.mark_bar(color=colors["eco-light-blue"], opacity=0.7).encode(y=alt.Y(
    "value:Q",
    sort=[],
    axis=alt.Axis(
        grid=False,
        title="Deal value (billion USD)",
        titleX=-5,
        titleY=-5,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="left",
        labelColor=colors["eco-light-blue"],
        titleColor=colors["eco-light-blue"],
        tickColor=colors["eco-light-blue"],
        domainColor=colors["eco-light-blue"],
        labelExpr="'$'+datum.value"
    ),
    scale=alt.Scale(domain=[0, 160]),
)).transform_filter("!((datum.Q=='2021Q3')|(datum.Q=='2021Q4'))")
bars2=base.mark_bar(color=colors["eco-light-blue"], opacity=0.4).encode(
    y=alt.Y(
        'value:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False
        ),
    scale=alt.Scale(domain=[0, 160]),
    ),
).transform_filter("(datum.Q=='2021Q3')|(datum.Q=='2021Q4')")
labels=base.mark_text(color=colors["eco-light-blue"], opacity=0.8, baseline='bottom',dy=-2).encode(
    y=alt.Y(
        'value:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False
        ),
    scale=alt.Scale(domain=[0, 160]),
    ),
    text='value:N'
)
layer1 = (((bars+bars2+labels+line).resolve_scale(
    y='independent'
).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
# layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

In [167]:
base = alt.Chart(f1).encode(x=alt.X(
    "Q:N",
    sort=[],
    axis=alt.Axis(
        # grid=False,
        title='',
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domain=False,
        tickCount=10,
        ticks=False,
        # orient="top",
        labelAngle=0,
        labelFont="Circular Std Book",
        offset=5,
        labelAlign='center',
        # labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
        labelExpr="slice(datum.value,0,4)=='2017'?'':slice(datum.value,0,4)=='2022'?'':indexof(datum.value,'Q1')>-1?slice(datum.value,0,4):slice(datum.value,4)",
    ),
))
points = base.mark_point(
    #  fillOpacity=0.8,
    color=colors['eco-turquiose'],
    strokeWidth=0.5,
    strokeOpacity=0.5,
    fill=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        ),size=150).encode(y=alt.Y(
    "deals:Q",
    sort=[],
    axis=alt.Axis(
        grid=False,
        title="Deal count",
        titleX=5,
        titleY=-5,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="right",
        labelColor=colors["eco-turquiose"],
        titleColor=colors["eco-turquiose"],
        tickColor=colors["eco-turquiose"],
        domainColor=colors["eco-turquiose"],
        titleFont="Circular Std Book",
        labelFont="Circular Std Book",
        offset=-21
    ),
    scale=alt.Scale(domain=[0, 1600]),
),
# tooltip=['deals','Q']
)
bars = base.mark_bar(
    fillOpacity=0.8,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        )
    ).encode(y=alt.Y(
    "value:Q",sort=[],axis=alt.Axis(labels=False,title='',domain=False,
        ticks=False,),
)).transform_filter("!((datum.Q=='2021Q3')|(datum.Q=='2021Q4'))")
bars2 = base.mark_bar(
    fillOpacity=0.8,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        )
    ).encode(y=alt.Y(
    "value:Q",
    sort=[],axis=alt.Axis(labels=False,title='',domain=False,
        ticks=False,),
    scale=alt.Scale(domain=[0, 160]),
)).transform_filter("((datum.Q=='2021Q3')|(datum.Q=='2021Q4'))")
bg=alt.Chart(pd.DataFrame([{'x':'2017Q4','y':170},{'x':'2022Q1','y':160},{'x':'2022Q2','y':160}])).mark_area(xOffset=-10,
    fillOpacity=0.4,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0.2),
            alt.GradientStop(color=colors["eco-orange"], offset=0.6),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=0.7,
        x2=1,
        y1=0.7,
        y2=0,
        )
    ).encode(x='x:N',y=alt.Y(
    "y:Q",
    
    sort=[],
    axis=alt.Axis(
        # grid=False,
        gridColor=colors["eco-light-blue"],
        gridOpacity=0.3,
        gridDash=[5,5],
        title="Deal value (billion USD)",
        titleX=-5,
        titleY=-5,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="left",
        labelColor=colors["eco-light-blue"],
        titleColor=colors["eco-light-blue"],
        tickColor=colors["eco-light-blue"],
        domainColor=colors["eco-light-blue"],
        titleFont="Circular Std Book",
        labelFont="Circular Std Book",
        labelExpr="'$'+datum.value"
    ),
    scale=alt.Scale(domain=[0, 160]),
))
labels=base.mark_text(color=colors["eco-light-blue"], opacity=0.8, baseline='bottom',dy=-2,font="Circular Std Book",size=11,dx=1).encode(
    y=alt.Y(
        'value:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False,
        domain=False,
        ticks=False,
        ),
    scale=alt.Scale(domain=[0, 160]),
    ),
    text='y:N'
).transform_calculate(y='round(datum.value)')
labels2=base.mark_text(color=colors["eco-turquiose"], opacity=0.8, baseline='bottom',dy=-8,font="Circular Std Book",size=11,dx=1).encode(
    y=alt.Y(
        'deals:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False,
        domain=False,
        ticks=False,
        ),
    scale=alt.Scale(domain=[0, 1600]),
    ),
    text='d:N'
).transform_calculate(d='round(datum.deals/1)')
layer1 = (((bg+bars+bars2+points+points).resolve_scale(
    y='independent'
).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + "_gradient.json")
# layer1.save("visualisation/" + f + "_gradient.png")
layer1

alt.LayerChart(...)

In [184]:
base = alt.Chart(f1).encode(x=alt.X(
    "Q:N",
    sort=[],
    axis=alt.Axis(
        # grid=False,
        title='',
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domain=False,
        tickCount=10,
        ticks=False,
        # orient="top",
        labelAngle=0,
        labelFont="Circular Std Book",
        offset=5,
        labelAlign='center',
        # labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
        labelExpr="slice(datum.value,0,4)=='2017'?'':slice(datum.value,0,4)=='2022'?'':indexof(datum.value,'Q1')>-1?slice(datum.value,0,4):slice(datum.value,4)",
    ),
))
points = base.mark_point(
    #  fillOpacity=0.8,
    color=colors['eco-turquiose'],
    strokeWidth=0.5,
    strokeOpacity=0.5,
    fill=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        ),size=150).encode(y=alt.Y(
    "deals:Q",
    sort=[],
    axis=alt.Axis(
        title="Deal count",
        titleX=-65,
        titleY=145,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="right",
        titleColor=colors["eco-turquiose"],
        titleFont="Circular Std Book",
        offset=-21,
        grid=False,
        labels=False,
        domain=False,
        ticks=False,
    ),
    scale=alt.Scale(domain=[0, 1600]),
),
# tooltip=['deals','Q']
)
bars = base.mark_bar(
    fillOpacity=0.8,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-light-blue"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        )
    ).encode(y=alt.Y(
    "value:Q",sort=[],axis=alt.Axis(labels=False,title='',domain=False,
        ticks=False,),
)).transform_filter("!((datum.Q=='2021Q3')|(datum.Q=='2021Q4'))")
bars2 = base.mark_bar(
    fillOpacity=0.8,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=0.9),
        ],
        x1=0,
        x2=1,
        y1=1,
        y2=1,
        )
    ).encode(y=alt.Y(
    "value:Q",
    sort=[],axis=alt.Axis(labels=False,title='',domain=False,
        ticks=False,),
    scale=alt.Scale(domain=[0, 160]),
)).transform_filter("((datum.Q=='2021Q3')|(datum.Q=='2021Q4'))")
bg=alt.Chart(pd.DataFrame([{'x':'2017Q4','y':160},{'x':'2022Q1','y':160},{'x':'2022Q2','y':160}])).mark_area(xOffset=-10,
    fillOpacity=0.4,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color='white', offset=0.2),
            alt.GradientStop(color=colors["eco-orange"], offset=0.6),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=0.7,
        x2=1,
        y1=0.7,
        y2=0,
        )
    ).encode(x='x:N',y=alt.Y(
    "y:Q",
    
    sort=[],
    axis=alt.Axis(
        # grid=False,
        title="Deal value (billion USD)",
        titleX=25,
        titleY=185,
        titleBaseline="bottom",
        titleAngle=0,
        titleAlign="left",
        titleColor=colors["eco-light-blue"],
        titleFont="Circular Std Book",
        grid=False,
        labels=False,
        domain=False,
        ticks=False,
    ),
    scale=alt.Scale(domain=[0, 160]),
))
labels=base.mark_text(color=colors["eco-light-blue"], opacity=0.8, baseline='bottom',dy=-2,font="Circular Std Book",size=11,dx=1).encode(
    y=alt.Y(
        'value:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False,
        domain=False,
        ticks=False,
        ),
    scale=alt.Scale(domain=[0, 160]),
    ),
    text='y:N'
).transform_calculate(y='round(datum.value)')
labels2=base.mark_text(color=colors["eco-turquiose"], opacity=0.8, baseline='bottom',dy=-8,font="Circular Std Book",size=11,dx=1).encode(
    y=alt.Y(
        'deals:Q',
        sort=[],
        axis=alt.Axis(
        grid=False,
        title='',
        labels=False,
        domain=False,
        ticks=False,
        ),
    scale=alt.Scale(domain=[0, 1600]),
    ),
    text='d:N'
).transform_calculate(d='round(datum.deals/1)')
layer1 = (((bg+bars+bars2+points+points+labels+labels2).resolve_scale(
    y='independent'
).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + "_gradient2.json")
# layer1.save("visualisation/" + f + "_gradient2.png")
layer1

alt.LayerChart(...)